In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from collections import namedtuple

class WebNavigator:
    def __init__(self):
        self.web_driver = webdriver.Chrome()
        self.web_driver.maximize_window()
        self.web_driver.implicitly_wait(10)
        self.web_driver.get("https://app-staging.nokodr.com/")

    def close_browser(self):
        if self.web_driver:
            self.web_driver.quit()

    def find_element_with_wait(self, locator, wait_time=10, is_clickable=False):
        wait = WebDriverWait(self.web_driver, wait_time)
        condition = EC.element_to_be_clickable if is_clickable else EC.presence_of_element_located
        return wait.until(condition(locator))

    def go_to_signup(self):
        signup_button = self.find_element_with_wait((By.LINK_TEXT, "Sign up"), is_clickable=True)
        signup_button.click()

    def enter_signup_details(self, email_address, accept_terms=True):
        email_input_field = self.find_element_with_wait(
            (By.XPATH, "(//input[@class='slds-input ng-untouched ng-pristine ng-valid'])[3]")
        )
        email_input_field.clear()
        email_input_field.send_keys(email_address)

        if accept_terms:
            terms_checkbox = self.find_element_with_wait((By.CLASS_NAME, "slds-checkbox__label"), is_clickable=True)
            terms_checkbox.click()

    def submit_registration(self):
        submit_button = self.find_element_with_wait(
            (By.XPATH, "//div[@class='slds-col slds-size_1-of-1']"), is_clickable=True
        )
        submit_button.click()

    def check_for_success(self, expected_success_text):
        try:
            success_element = self.find_element_with_wait((By.XPATH, "//*[contains(text(),'OTP')]"))
            actual_text = success_element.text
            if expected_success_text in actual_text:
                print(f"Verification Successful: Found '{expected_success_text}' in success message")
            else:
                print(f"Verification Failed: Expected '{expected_success_text}', but got '{actual_text}'")
        except TimeoutException:
            print("Verification Failed: Success message did not appear within the time limit")

    def check_for_error(self, expected_error_message):
        try:
            
            WebDriverWait(self.web_driver, 2).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'OTP')]"))
            )
            print(f"Error: Success message appeared when expecting error '{expected_error_message}'")
        except TimeoutException:
           
            error_elements = self.web_driver.find_elements(By.CLASS_NAME, "error-message")
            error_messages = [error.text for error in error_elements if error.text]
            if any(expected_error_message in message for message in error_messages):
                print(f"Error Verification Successful: Found '{expected_error_message}' in error messages")
            else:
                print(f"Error Verification Failed: Expected '{expected_error_message}', but found {error_messages or 'no errors'}")

class SignupTester:
    def execute_test(self, email, terms_checked, scenario_type, expected_outcome):
        navigator = WebNavigator()
        try:
            navigator.go_to_signup()
            navigator.enter_signup_details(email, terms_checked)
            navigator.submit_registration()

            if scenario_type == "success":
                navigator.check_for_success(expected_outcome)
            elif scenario_type == "error":
                navigator.check_for_error(expected_outcome)
        except Exception as error:
            print(f"Test encountered an error: {error}")
        finally:
            navigator.close_browser()


TestScenario = namedtuple('TestScenario', ['test_name', 'email', 'terms_checked', 'scenario_type', 'expected_outcome'])

test_scenarios = [
    TestScenario(
        test_name="Successful Registration",
        email="valid.email@example.com",
        terms_checked=True,
        scenario_type="success",
        expected_outcome="OTP"
    ),
    TestScenario(
        test_name="Invalid Email Format",
        email="invalid-email",
        terms_checked=True,
        scenario_type="error",
        expected_outcome="Invalid email format"
    ),
    TestScenario(
        test_name="Missing Email",
        email="",
        terms_checked=True,
        scenario_type="error",
        expected_outcome="Email is required"
    ),
    TestScenario(
        test_name="Terms Not Accepted",
        email="valid.email@example.com",
        terms_checked=False,
        scenario_type="error",
        expected_outcome="You must accept the terms and conditions"
    )
]

if __name__ == "__main__":
    tester = SignupTester()
    for scenario in test_scenarios:
        print(f"\nExecuting test: {scenario.test_name}...")
        tester.execute_test(
            email=scenario.email,
            terms_checked=scenario.terms_checked,
            scenario_type=scenario.scenario_type,
            expected_outcome=scenario.expected_outcome
        )



Executing test: Successful Registration...
Verification Successful: Found 'OTP' in success message

Executing test: Invalid Email Format...
Error: Success message appeared when expecting error 'Invalid email format'

Executing test: Missing Email...
Error: Success message appeared when expecting error 'Email is required'

Executing test: Terms Not Accepted...
Error: Success message appeared when expecting error 'You must accept the terms and conditions'
